# Imports and opening the file

In [1]:
# imports

import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm')

import datapurifier as dp
from datapurifier import Mleda, Nlpurifier, NLAutoPurifier, MlReport, Nlpeda
from collections import Counter

from nltk.corpus import stopwords
StopWords = stopwords.words('english')

C:\Users\Shark\anaconda3\lib\site-packages\spacy\util.py:758: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.1.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
# open file in df

path = "data/data.csv"
full_data_df = pd.read_csv(path)
full_data_df.columns=["text"]
print(full_data_df.shape)

# create small df for testing
data_df = full_data_df.head(40000)
data_df.columns=["text"]

# print first 10 lines for inspection
data_df.head(10)

(2811773, 1)


,text
0,@sprintcare and how do you propose we do that
1,@sprintcare I have sent several private messag...
2,@115712 Please send us a Private Message so th...
3,@sprintcare I did.
4,@115712 Can you please send us a private messa...
5,@sprintcare is the worst customer service
6,@115713 This is saddening to hear. Please shoo...
7,@sprintcare You gonna magically change your co...
8,@115713 We understand your concerns and we'd l...
9,@sprintcare Since I signed up with you....Sinc...


# Pre-preprocessing

The dataset is too large for my computer to handle, but I did not want to lose valuable information by simply cutting it in half. So before I started preprocessing with Spacy and other heavy modules, I manually split the data into words and found the most frequent keywords. I extracted all instances that contain the found keywords to shorten the data. This step is also helpful to already create a dataset focused on a few common topics, which will make it easier for the clustering algorithm to find clusters. 

In [3]:
# shorten and precluster data by frequent keywords

# open file
path = "data/data.csv"
with open(path, "r", encoding='utf-8') as infile:
    raw_content = infile.read()
    
# preclean and split content
new_content = raw_content.replace('\n', '. ')
new_content = new_content.replace('..', '.')
new_content = new_content.replace('  ', ' ')
new_content = new_content.replace('"', '')
split_content = new_content.split()

# get most frequent keywords
freq_counts = Counter(list(split_content))

# remove stopwords from frequency dict
for word in StopWords:
    if word in freq_counts:
        del freq_counts[word]
        
most_common = freq_counts.most_common(200)

# get 100 most common words in list for viewing
most_common_words = []
for item in most_common:
    most_common_words.append(item[0])

In [4]:
# view most common words
print(most_common_words)

['I', 'us', 'DM', 'Please', 'We', 'get', 'Hi', 'like', 'please', '@AmazonHelp', 'look', 'know', 'you.', 'sorry', 'help', "I'm", '-', 'Thanks', 'send', '.', 'Can', 'service', 'need', 'phone', 'would', 'If', '@AppleSupport', 'Thank', 'email', 'account', 'number', "We're", '&amp;', 'see', 'back', 'still', 'hear', 'You', 'want', 'take', 'team', 'here:', 'time', 'help.', 'check', 'let', 'The', 'order', 'issue', 'via', 'one', 'Hey', 'address', 'happy', 'details', 'Sorry', 'This', "we'll", 'new', 'able', 'What', 'got', 'make', 'going', 'https://t.co/GDrqU22YpT.', 'contact', 'this.', 'flight', 'customer', 'It', 'assist', "We'll", "We'd", '2', '@AmericanAir', 'Hi,', 'My', '@Uber_Support', 'sure', 'link', 'getting', 'go', 'work', 'I’m', 'use', 'call', 'full', 'it.', 'store', 'Have', 'No', 'delivery', 'could', 'Is', '@Delta', "can't", 'try', 'working', 'de', 'Send', 'love', 'even', 'app', 'provide', 'iPhone', 'update', 'reach', 'it’s', 'follow', '@VirginTrains', 'How', 'using', 'iOS', 'issues', "

In [5]:
# define keywords to filter data before heavy preprocessing

keywords = ['phone', 'email', 'account', 'number', 'team', 'order', 'issue', 'address', 'details',
           'contact', 'flight', 'link', 'booking', 'baggage', 'call', 'cancel', 'delivery', 'app', 'update', 'iphone', 'refund', 'due']

In [6]:
# get instances into list
instance_list = data_df['text'].tolist()

# split costomer questions from employee replies
customer_instances = []

for item in instance_list:
    if item[1].isalpha() == True:
        customer_instances.append(item)

# filter instances by keyword
filtered_instances = []

for instance in customer_instances:
    split_instance = instance.split()
    for word in split_instance:
        if word.lower() in keywords:
            filtered_instances.append(instance)
            break

# join for further preprocessing            
joined_filtered_data = '. '.join(filtered_instances)
len(joined_filtered_data)

514521

# Pre-processing and cleaning

Now that we have a filtered, shortened, dataset, we can process it with SpaCy and DataPurifier. After cleaning, the data is saved to use in the clustering pipelines. The following cleaning steps are performed:

- Lemmatize
- Lowercase
- Remove stop words
- Remove special characters, numeric characters and punctuation
- Remove links and tokens starting with '@'
- Remove tokens starting with a '-' or '^' as those are employee names
- Strip leading, trailing and duplicate whitespaces
- Remove most frequent and least frequent words with a threshold of 10

In [7]:
# load filtered data in spacy

# shorten data to spacy max length
joined_filtered_data = joined_filtered_data[:100000]
doc = nlp(joined_filtered_data)

# get all spacy sentences in list
sents = []

for sent in doc.sents:
    sent_list = []
    for token in sent:
        token = str(token)
        # removing tokens that datapurifier will not remove
        if token.startswith('-') == False and token.startswith('^') == False and token.startswith('@') == False and token != " " and token != "" and token != '\n':
            sent_list.append(token)
    sent_list = ' '.join(sent_list)
    sents.append(sent_list)
    
# remove super short sentences
sentences = []
for item in sents:
    if len(item) > 3:
        sentences.append(item)
        
# get sentences in dataframe for further processing
d = {'sentences': sentences}
filtered_df = pd.DataFrame(d)
filtered_df.shape

(1582, 1)

In [8]:
# clean data with DataPurifier
cleaned_df = Nlpurifier(filtered_df, "sentences")

GridspecLayout(children=(Checkbox(value=False, description='Drop Null Rows', indent=False, layout=Layout(grid_…


Convert Word to its Base Form


interactive(children=(RadioButtons(description='Technique:', options=('None', 'Stemming', 'Lemmatization'), va…

Remove Top Common Words


interactive(children=(Checkbox(value=False, description='Remove Top Common Words'), Output()), _dom_classes=('…

Remove Top Rare Words


interactive(children=(Checkbox(value=False, description='Remove Top Rare Words'), Output()), _dom_classes=('wi…

Button(description='Start Purifying', style=ButtonStyle())

In [11]:
# preview cleaned df
clean_df = cleaned_df.df
clean_df['raw_sentences'] = sentences # add original sentences

clean_df.head(10)

,sentences,raw_sentences
0,like email copy spectrum update training,Would you like me to email you a copy of one s...
1,correct way ocs account takeover email consent...,The correct way to do it is via an OCS Account...
2,actually break link send incorrect information,actually that 's a broken link you sent me and...
3,,How ?
4,phone number email,"I have my phone number and email , that 's it ."
5,equipment service,How did I get equipment and service ?
6,literally try pay find,I 'm literally trying to pay and nobody can fi...
7,thank resolve issue quickly,Thank you for resolving my issue so quickly ! !
8,y all good fanforlife,Y’all are the best ☺ ️ # fanforlife .
9,frustrated order dinner saturday app,So frustrated with 😡 Ordered dinner on Saturda...


In [10]:
# save filtered and cleaned df for use in other notebooks
# clean_df.to_csv("data/cleaned_customer_dataframe.csv")